In [50]:
from doiref import CrossRefClient

session = CrossRefClient()

ref = session.doi2json('http://dx.doi.org/10.1109/3DV50981.2020.00103')
ref

{'indexed': {'date-parts': [[2024, 10, 29]],
  'date-time': '2024-10-29T09:11:08Z',
  'timestamp': 1730193068708,
  'version': '3.28.0'},
 'reference-count': 28,
 'publisher': 'IEEE',
 'license': [{'start': {'date-parts': [[2020, 11, 1]],
    'date-time': '2020-11-01T00:00:00Z',
    'timestamp': 1604188800000},
   'content-version': 'vor',
   'delay-in-days': 0,
   'URL': 'https://ieeexplore.ieee.org/Xplorehelp/downloads/license-information/IEEE.html'},
  {'start': {'date-parts': [[2020, 11, 1]],
    'date-time': '2020-11-01T00:00:00Z',
    'timestamp': 1604188800000},
   'content-version': 'stm-asf',
   'delay-in-days': 0,
   'URL': 'https://doi.org/10.15223/policy-029'},
  {'start': {'date-parts': [[2020, 11, 1]],
    'date-time': '2020-11-01T00:00:00Z',
    'timestamp': 1604188800000},
   'content-version': 'stm-asf',
   'delay-in-days': 0,
   'URL': 'https://doi.org/10.15223/policy-037'}],
 'content-domain': {'domain': [], 'crossmark-restriction': False},
 'published-print': {'date

In [2]:
corresponding = ['Meiler']

In [51]:
def formatSPP(ref):
    s = ""
    preprint = False
    for author in ref['author']:
        s += author['given'][0] + ". " + author['family'] + ","
        if author['family'] in corresponding:
            s += '*'
        s += ' '
    s = s[:-1] + '\n'
    s += ref['title'] + '\n'
    if 'container-title-short' in ref:
        s += ref['container-title-short'] + " "
    elif 'container-title' in ref and ref['container-title']:
        s += ref['container-title'] + " "
    else:
        if ref['publisher'] == 'Cold Spring Harbor Laboratory':
            s += 'bioRxiv '
            preprint = True
        elif ref['publisher'] == 'American Chemical Society (ACS)':
            s += 'ChemRxiv '
            preprint = True
        elif ref['publisher'] == 'arXiv':
            s += 'arXiv '
            preprint = True
        else:
            raise KeyError('container-title-short')
    if 'created' in ref:
        s += str(ref['created']['date-parts'][0][0])
    elif 'issued' in ref:
        s += str(ref['issued']['date-parts'][0][0])
    else:
        raise KeyError('year')
    if not preprint:
        if 'volume' in ref : s += ", " + str(ref['volume'])
        if 'page' in ref : s += ", " + str(ref['page'])
    s += ". " + ref['URL']
    return s


In [4]:
all_dois = """https://doi.org/10.1101/2024.08.06.606224 

https://doi.org/10.48550/arXiv.2411.09820 

https://doi.org/10.1101/2024.06.20.599843 

https://doi.org/10.26434/chemrxiv-2024-hr692 

https://doi.org/10.1021/acschemneuro.4c00086 

https://doi.org/10.1021/acssynbio.3c00753 

https://doi.org/10.1371/journal.pcbi.1011939 

https://doi.org/10.1021/acs.jctc.3c01081 

https://doi.org/10.1101/2022.11.15.516482  

https://doi.org/10.1016/j.sbi.2023.102645

https://doi.org/10.1609/aaai.v37i12.26679 

https://doi.org/10.1016/j.bpj.2022.11.011 

https://doi.org/10.1101/2023.04.17.537185 

https://doi.org/10.3389/fmolb.2023.1121962 

https://doi.org/10.1016/j.str.2022.11.012 

https://doi.org/10.1073/pnas.2206129119 

https://doi.org/10.1158/0008-5472.CAN-21-3798 

https://doi.org/10.7554/eLife.75751

https://doi.org/10.3389/fphar.2022.833099 

https://doi.org/10.1021/acs.jproteome.1c00410 

https://doi.org/10.1021/acs.jcim.0c01001 

https://doi.org/10.1021/acs.jcim.0c01001 

https://doi.org/10.1109/3DV50981.2020.00103

https://doi.org/10.48550/arXiv.2007.07029 

https://doi.org/10.1007/s10822-019-00199-8 

https://doi.org/10.3390/molecules22091576 

https://doi.org/10.1016/j.bmcl.2016.07.071

https://doi.org/10.1007/s10822-016-9895-2 

https://doi.org/10.1007/s10822-015-9893-9 

https://doi.org/10.1124/pr.112.007336 

https://doi.org/10.3390/molecules18010735 

https://doi.org/10.3390/molecules17089971 

https://doi.org/10.1109/CIBCB.2012.6217248

https://doi.org/10.1109/CIBCB.2012.6217246

https://doi.org/10.1109/CIBCB.2011.5948478

https://doi.org/10.1002/cmdc.201100510 

https://doi.org/10.1109/CIBCB.2011.5948478

https://doi.org/10.1016/j.bpj.2010.12.2344 

https://doi.org/10.1016/j.bmcl.2010.07.061

https://doi.org/10.1109/CIBCB.2009.4925736

https://doi.org/10.1109/CIBCB.2009.4925709

https://doi.org/10.1002/qsar.200330837 

https://doi.org/10.1023/A:1023060720156 

https://doi.org/10.1006/jmre.2002.2599

https://doi.org/10.1021/ci0102970 

https://doi.org/10.1021/ci000021c

https://doi.org/10.1007/PL00010287"""

In [5]:
from tqdm import tqdm

all_refs = []

for doi in tqdm(all_dois.split()):
    all_refs.append(session.doi2json(doi))

  0%|          | 0/47 [00:00<?, ?it/s]

100%|██████████| 47/47 [00:25<00:00,  1.84it/s]


In [52]:
errors = []

for ref in all_refs:
    try:
        if 'created' in ref:
            if ref['created']['date-parts'][0][0] < 2020:
                continue
        elif 'issued' in ref:
            if ref['issued']['date-parts'][0][0] < 2020:
                continue
        print(formatSPP(ref))
    except KeyError as e:
        errors.append([e.args[0], ref['URL'], ref['publisher']])
print(errors)

C. Moth, J. Sheehan, A. Mamun, R. Sivley, A. Gulsevin, D. Rinker, J. Capra, J. Meiler,*
VUStruct: a compute pipeline for high throughput and personalized structural biology
bioRxiv 2024. http://dx.doi.org/10.1101/2024.08.06.606224
M. Ertelt, R. Moretti, J. Meiler,* C. Schoeder,
Self-supervised machine learning methods for protein design improve sampling, but not the identification of high-fitness variants
bioRxiv 2024. http://dx.doi.org/10.1101/2024.06.20.599843
J. DeCorte, B. Brown, J. Meiler,*
Interpretable deep-learning pKa prediction for small molecule drugs via atomic sensitivity analysis
ChemRxiv 2024. http://dx.doi.org/10.26434/chemrxiv-2024-hr692
C. Jones, B. Brown, D. Schultz, J. Engers, V. Kramlinger, J. Meiler,* C. Lindsley,
Computer-Aided Design and Biological Evaluation of Diazaspirocyclic D<sub>4</sub>R Antagonists
ACS Chem. Neurosci. 2024, 15, 2396-2407. http://dx.doi.org/10.1021/acschemneuro.4c00086
M. Ertelt, J. Meiler,* C. Schoeder,
Combining Rosetta Sequence Design w